In [1]:
from __future__ import print_function, division
import sys
import matplotlib.pyplot as plt
import numpy as np
import pickle
import glob
from music21 import converter, instrument, note, chord, stream
from keras.layers import Input, Dense, Reshape, Dropout, CuDNNLSTM, Bidirectional
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []

    for file in glob.glob("Pokemon MIDIs/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
            
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

In [11]:
def prepare_sequences(notes, n_vocab, seq_length):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = seq_length

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab)/2) / (float(n_vocab)/2)
    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [4]:
def generate_notes(model, network_input, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)
    
    # Get pitch names and store in a dictionary
    pitchnames = sorted(set(item for item in notes))
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        
        pattern = numpy.append(pattern,index)
        #pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [5]:
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

In [9]:
class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        
        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates note sequences
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(generated_seq)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_discriminator(self):

        model = Sequential()
        model.add(CuDNNLSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(CuDNNLSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        seq = Input(shape=self.seq_shape)
        validity = model(seq)

        return Model(seq, validity)
      
    def build_generator(self):

        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)

        return Model(noise, seq)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load and convert the data
        notes = get_notes()
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab, self.seq_length)

        # Adversarial ground truths
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        # Training the model
        for epoch in range(epochs):

            # Training the discriminator
            # Select a random batch of note sequences
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]

            #noise = np.random.choice(range(484), (batch_size, self.latent_dim))
            #noise = (noise-242)/242
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new note sequences
            gen_seqs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            #  Training the Generator
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as real)
            g_loss = self.combined.train_on_batch(noise, real)

            # Print the progress and save into loss lists
            if epoch % sample_interval == 0:
              print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
              self.disc_loss.append(d_loss[0])
              self.gen_loss.append(g_loss)
        
        self.generate(notes)
        self.plot_loss()
        
    def generate(self, input_notes):
        # Get pitch names and store in a dictionary
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        
        # Use random noise to generate sequences
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        
        pred_notes = [x*242+242 for x in predictions[0]]
        pred_notes = [int_to_note[int(x)] for x in pred_notes]
        
        create_midi(pred_notes, 'gan_final')
        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_final.png', transparent=True)
        plt.close()

In [12]:
gan = GAN(rows=50)    
gan.train(epochs=1000, batch_size=32, sample_interval=1)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_7 (CuDNNLSTM)     (None, 50, 512)           1054720   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 1024)              4202496   
_________________________________________________________________
dense_22 (Dense)             (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)                

Parsing Pokemon MIDIs\Pokemon DiamondPearlPlatinum - Poketch Addition.mid
Parsing Pokemon MIDIs\Pokemon DiamondPearlPlatinum - Research Lab.mid
Parsing Pokemon MIDIs\Pokemon DiamondPearlPlatinum - Route 202.mid
Parsing Pokemon MIDIs\Pokemon DiamondPearlPlatinum - Route 205 Night.mid
Parsing Pokemon MIDIs\Pokemon DiamondPearlPlatinum - Route 205.mid
Parsing Pokemon MIDIs\Pokemon DiamondPearlPlatinum - Route 209.mid
Parsing Pokemon MIDIs\Pokemon DiamondPearlPlatinum - Route 225 Night.mid
Parsing Pokemon MIDIs\Pokemon DiamondPearlPlatinum - Route 228 Night.mid
Parsing Pokemon MIDIs\Pokemon DiamondPearlPlatinum - Sandgem Town Day.mid
Parsing Pokemon MIDIs\Pokemon DiamondPearlPlatinum - Spear Pillar.mid
Parsing Pokemon MIDIs\Pokemon DiamondPearlPlatinum - Stark Mountain.mid
Parsing Pokemon MIDIs\Pokemon DiamondPearlPlatinum - Team Galactic Grunt Battle.mid
Parsing Pokemon MIDIs\Pokemon DiamondPearlPlatinum - Torn WorldDistortion World.mid
Parsing Pokemon MIDIs\Pokemon DiamondPearlPlatinum -

Parsing Pokemon MIDIs\Pokemon Mystery Dungeon Explorers of TimeDarknessSky - I Dont Want to Say Goodbye.mid
Parsing Pokemon MIDIs\Pokemon Mystery Dungeon Explorers of TimeDarknessSky - In the Hands of Fate.mid
Parsing Pokemon MIDIs\Pokemon Mystery Dungeon Explorers of TimeDarknessSky - Lower Brine Cave.mid
Parsing Pokemon MIDIs\Pokemon Mystery Dungeon Explorers of TimeDarknessSky - Oh No.mid
Parsing Pokemon MIDIs\Pokemon Mystery Dungeon Explorers of TimeDarknessSky - Palkias Onslaught.mid
Parsing Pokemon MIDIs\Pokemon Mystery Dungeon Explorers of TimeDarknessSky - Pelippers Island.mid
Parsing Pokemon MIDIs\Pokemon Mystery Dungeon Explorers of TimeDarknessSky - Sealed Ruin Pit.mid
Parsing Pokemon MIDIs\Pokemon Mystery Dungeon Explorers of TimeDarknessSky - Sky Peak Final Pass.mid
Parsing Pokemon MIDIs\Pokemon Mystery Dungeon Explorers of TimeDarknessSky - Spacial Cliffs.mid
Parsing Pokemon MIDIs\Pokemon Mystery Dungeon Explorers of TimeDarknessSky - Spindas Cafe.mid
Parsing Pokemon MIDI

C:\Users\Michael\Anaconda3\envs\CITS4404\lib\site-packages\keras\engine\training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.697306, acc.: 0.00%] [G loss: 0.690526]
1 [D loss: 0.678759, acc.: 70.31%] [G loss: 0.690840]


...

301 [D loss: 0.684163, acc.: 56.25%] [G loss: 0.709775]
302 [D loss: 0.677682, acc.: 62.50%] [G loss: 0.708760]
303 [D loss: 0.705878, acc.: 48.44%] [G loss: 0.702957]
304 [D loss: 0.677360, acc.: 60.94%] [G loss: 0.705285]
305 [D loss: 0.675259, acc.: 59.38%] [G loss: 0.704273]
306 [D loss: 0.701075, acc.: 50.00%] [G loss: 0.706806]
307 [D loss: 0.705814, acc.: 51.56%] [G loss: 0.702416]
308 [D loss: 0.670498, acc.: 60.94%] [G loss: 0.702546]
309 [D loss: 0.698523, acc.: 43.75%] [G loss: 0.707081]
310 [D loss: 0.690883, acc.: 48.44%] [G loss: 0.697638]
311 [D loss: 0.693458, acc.: 48.44%] [G loss: 0.716147]
312 [D loss: 0.684393, acc.: 54.69%] [G loss: 0.707771]
313 [D loss: 0.678409, acc.: 57.81%] [G loss: 0.701038]
314 [D loss: 0.701087, acc.: 46.88%] [G loss: 0.714053]
315 [D loss: 0.691648, acc.: 50.00%] [G loss: 0.710550]
316 [D loss: 0.692114, acc.: 51.56%] [G loss: 0.702774]
317 [D loss: 0.693039, acc.: 53.12%] [G loss: 0.703363]
318 [D loss: 0.694629, acc.: 51.56%] [G loss: 0.

448 [D loss: 0.692948, acc.: 46.88%] [G loss: 0.705852]
449 [D loss: 0.699201, acc.: 43.75%] [G loss: 0.702405]
450 [D loss: 0.699684, acc.: 34.38%] [G loss: 0.708785]
451 [D loss: 0.696102, acc.: 53.12%] [G loss: 0.709278]
452 [D loss: 0.689787, acc.: 54.69%] [G loss: 0.709291]
453 [D loss: 0.693151, acc.: 48.44%] [G loss: 0.706346]
454 [D loss: 0.694557, acc.: 45.31%] [G loss: 0.708895]
455 [D loss: 0.695004, acc.: 43.75%] [G loss: 0.713265]
456 [D loss: 0.693116, acc.: 54.69%] [G loss: 0.711931]
457 [D loss: 0.693577, acc.: 50.00%] [G loss: 0.704899]
458 [D loss: 0.696520, acc.: 51.56%] [G loss: 0.710417]
459 [D loss: 0.694575, acc.: 45.31%] [G loss: 0.712063]
460 [D loss: 0.691768, acc.: 45.31%] [G loss: 0.712171]
461 [D loss: 0.694856, acc.: 42.19%] [G loss: 0.713932]
462 [D loss: 0.698966, acc.: 43.75%] [G loss: 0.711519]
463 [D loss: 0.692313, acc.: 48.44%] [G loss: 0.708075]
464 [D loss: 0.693832, acc.: 45.31%] [G loss: 0.709201]
465 [D loss: 0.695414, acc.: 50.00%] [G loss: 0.

597 [D loss: 0.693890, acc.: 48.44%] [G loss: 0.699118]
598 [D loss: 0.693841, acc.: 53.12%] [G loss: 0.700178]
599 [D loss: 0.694766, acc.: 54.69%] [G loss: 0.698219]
600 [D loss: 0.694260, acc.: 45.31%] [G loss: 0.697247]
601 [D loss: 0.692740, acc.: 53.12%] [G loss: 0.697831]
602 [D loss: 0.693183, acc.: 48.44%] [G loss: 0.698674]
603 [D loss: 0.693568, acc.: 50.00%] [G loss: 0.696407]
604 [D loss: 0.694341, acc.: 45.31%] [G loss: 0.696921]
605 [D loss: 0.694589, acc.: 37.50%] [G loss: 0.697273]
606 [D loss: 0.693855, acc.: 45.31%] [G loss: 0.698061]
607 [D loss: 0.693404, acc.: 53.12%] [G loss: 0.697068]
608 [D loss: 0.694996, acc.: 39.06%] [G loss: 0.696865]
609 [D loss: 0.692595, acc.: 54.69%] [G loss: 0.696261]
610 [D loss: 0.693153, acc.: 48.44%] [G loss: 0.695293]
611 [D loss: 0.692366, acc.: 54.69%] [G loss: 0.695899]
612 [D loss: 0.693592, acc.: 42.19%] [G loss: 0.694639]
613 [D loss: 0.693544, acc.: 53.12%] [G loss: 0.697603]
614 [D loss: 0.692099, acc.: 51.56%] [G loss: 0.

745 [D loss: 0.693102, acc.: 45.31%] [G loss: 0.695645]
746 [D loss: 0.691685, acc.: 48.44%] [G loss: 0.696168]
747 [D loss: 0.690920, acc.: 56.25%] [G loss: 0.695405]
748 [D loss: 0.695504, acc.: 50.00%] [G loss: 0.694356]
749 [D loss: 0.696446, acc.: 43.75%] [G loss: 0.694669]
750 [D loss: 0.690715, acc.: 50.00%] [G loss: 0.693757]
751 [D loss: 0.693760, acc.: 46.88%] [G loss: 0.695034]
752 [D loss: 0.693100, acc.: 56.25%] [G loss: 0.695288]
753 [D loss: 0.692969, acc.: 50.00%] [G loss: 0.694506]
754 [D loss: 0.693032, acc.: 56.25%] [G loss: 0.694663]
755 [D loss: 0.692319, acc.: 53.12%] [G loss: 0.695091]
756 [D loss: 0.693936, acc.: 50.00%] [G loss: 0.694843]
757 [D loss: 0.693869, acc.: 50.00%] [G loss: 0.694732]
758 [D loss: 0.692513, acc.: 53.12%] [G loss: 0.693874]
759 [D loss: 0.694362, acc.: 46.88%] [G loss: 0.696176]
760 [D loss: 0.693945, acc.: 46.88%] [G loss: 0.695969]
761 [D loss: 0.692860, acc.: 50.00%] [G loss: 0.697619]
762 [D loss: 0.692528, acc.: 53.12%] [G loss: 0.

893 [D loss: 0.691224, acc.: 59.38%] [G loss: 0.698221]
894 [D loss: 0.692094, acc.: 57.81%] [G loss: 0.698463]
895 [D loss: 0.692227, acc.: 56.25%] [G loss: 0.698214]
896 [D loss: 0.694839, acc.: 54.69%] [G loss: 0.697037]
897 [D loss: 0.692653, acc.: 50.00%] [G loss: 0.696693]
898 [D loss: 0.693347, acc.: 51.56%] [G loss: 0.699200]
899 [D loss: 0.693864, acc.: 37.50%] [G loss: 0.697201]
900 [D loss: 0.692615, acc.: 51.56%] [G loss: 0.697744]
901 [D loss: 0.693577, acc.: 53.12%] [G loss: 0.696811]
902 [D loss: 0.691587, acc.: 51.56%] [G loss: 0.696663]
903 [D loss: 0.692979, acc.: 45.31%] [G loss: 0.698012]
904 [D loss: 0.693904, acc.: 40.62%] [G loss: 0.697274]
905 [D loss: 0.693742, acc.: 43.75%] [G loss: 0.697378]
906 [D loss: 0.693260, acc.: 48.44%] [G loss: 0.697222]
907 [D loss: 0.691517, acc.: 56.25%] [G loss: 0.696316]
908 [D loss: 0.693282, acc.: 50.00%] [G loss: 0.696676]
909 [D loss: 0.695362, acc.: 50.00%] [G loss: 0.694923]
910 [D loss: 0.693833, acc.: 48.44%] [G loss: 0.